In [2]:
import neurokit2 as nk
import numpy as np
import pandas as pd
from typing import List


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# EMG imports:
from backbone import Subject
from timesrsutils import TimeSrsTools

def callfunc():
    ids = ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
    subjects_emg: dict[str, Subject] = {}
    for subject_id in ids:
        # Note: data folder should exist
        path = os.path.join("local/data", f"BSL-EMG-Book{subject_id}.csv")
        # where padding
        subj = Subject.from_csv(name=subject_id, path=path, fs=FS_EMG, target_len=EMG_LEN)
        subjects_emg[subject_id] = subj

    # check load results
    print(subjects_emg.keys())

    # example with "P2"
    print(subjects_emg["P2"].df.info())
    s = subjects_emg["P2"]
    TimeSrsTools.emg_preprocess_hilbert(s)
    s.df.to_csv("output.csv", index=False)

callfunc()

In [15]:
# load csv process:

ids = ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
raw_emg = []
for subject_id in ids:
    raw_emg.append(pd.read_csv(f"local/data\\BSL-EMG-Book{subject_id}.csv", delimiter=";", encoding="utf-8"))

#path = "C:\\Users\\User\\Documents\\P\\ibe\\2025a\\BmSP-online\\BSP_project\\BSL-EMG-BookP2.csv"

#for a single df in the list:

#df = pd.read_csv(path, delimiter=";", encoding="utf-8") #EMG in mV
df = df.replace(',', '.', regex=True)
target_len=60000

current_len = len(df)
if current_len < target_len:
    pad_size = target_len - current_len
    zeros = np.zeros(pad_size)
    emg_padded = np.concatenate([df['EMG'].values, zeros])
    int_emg_padded = np.concatenate([df['Integrated EMG'].values, zeros])
    df = pd.DataFrame({'EMG': emg_padded, 'Integrated EMG': int_emg_padded})
#elif current_length > target_length:
#    df = df.iloc[:target_length].reset_index(drop=True)
#


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   EMG             60000 non-null  object
 1   Integrated EMG  60000 non-null  object
dtypes: object(2)
memory usage: 937.6+ KB


In [16]:
df.head(10)

,EMG,Integrated EMG
0,-0.00183105,0.00370483
1,0.022583,0.00393066
2,0.0238037,0.0041687
3,0.0134277,0.00430298
4,-0.0109863,0.00441284
5,0.0402832,0.00481567
6,0.0610352,0.00542603
7,0.0140381,0.00556641
8,0.00183105,0.00558472
9,-0.0146484,0.0057312


In [ ]:
def window_dataframe(dataframe: pd.DataFrame, window_size: int, overlap: int = 0) -> List[pd.DataFrame]:
    if window_size <= 0:
        raise ValueError("window_size must be > 0")

    if overlap < 0:
        raise ValueError("overlap must be >= 0")

    if overlap >= window_size:
        raise ValueError("overlap must be smaller than window_size")

    step = window_size - overlap
    windows = []

    for start in range(0, len(dataframe) - window_size + 1, step):
        end = start + window_size
        window = dataframe.iloc[start:end].copy()
        windows.append(window)

    return windows

windows = window_dataframe(raw_emg[0], window_size=1000, overlap=500)

print(len(raw_emg[0]))
print(len(windows))
for window in windows:
    print(window.columns, len(window))

In [18]:
ids = ['P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
raw_emg = []
for subject_id in ids:
    raw_emg.append(pd.read_csv(f"local/data\\BSL-EMG-Book{subject_id}.csv", encoding="utf-8"))

In [20]:
df_raw = pd.DataFrame(columns=ids)

In [22]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   P2      0 non-null      object
 1   P3      0 non-null      object
 2   P4      0 non-null      object
 3   P5      0 non-null      object
 4   P6      0 non-null      object
 5   P7      0 non-null      object
 6   P8      0 non-null      object
 7   P9      0 non-null      object
 8   P10     0 non-null      object
dtypes: object(9)
memory usage: 124.0+ bytes


In [ ]:
len(raw_emg)
for e in raw_emg:
    print(len(e[0]))
 # if len e is less then max of raw_emg length, fill with zeros to the same size

In [ ]:
def check_time(subject):
# measure length of mV data, divide by fs = 500 Hz (samples / samples/seconds = seconds)